# Battery Storage Problem analysis

In [ ]:
%load_ext autoreload
%autoreload 2

%cd ../

In [ ]:
import itertools
import json
import os
import pickle

import matplotlib.legend
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so

import run_storage

sns.set_style('darkgrid')

SEEDS = range(10)

savedir = 'analysis/plots'
os.makedirs(savedir, exist_ok=True)

In [ ]:
def cvar(x: np.ndarray, q: float) -> float:
    # TODO: check numpy's quantile function
    return np.mean(x[x >= np.quantile(x, q)]).item()

# Optimal

In [ ]:
with np.load(f'out/storage_mlp_shuffle/optimal_task_loss.npz') as npz:
    optimal_task_loss = npz['task_loss'].mean()
    sd = npz['task_loss'].std()
    # print(npz['task_loss'])
print(f'Optimal task loss: {optimal_task_loss:.3f} ± {sd:.3f}')

# Pre-trained only

In [ ]:
rows = []
task_losses = {}

for s in SEEDS:
    with np.load(f'out/storage_mlp_shuffle/preds_s{s}.npz') as npz:
        for δ in np.linspace(0, 1, endpoint=True, num=101):
            row = {
                'seed': s,
                'delta': δ,
                'CVaR': cvar(npz['task_loss'], q=δ.item()),
            }
            rows.append(row)
        task_losses[s] = npz['task_loss'].mean()

df_pretrained = pd.DataFrame(rows)
sr_pretrained = pd.Series(task_losses)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), tight_layout=True)
sns.boxplot(sr_pretrained, ax=ax)

In [ ]:
(
    so.Plot(df_pretrained, x='delta', y='CVaR', color='seed')
    .add(so.Line())
    .scale(color=so.Nominal())
)

In [ ]:
def plot_ecdf(ax: plt.Axes, data: np.ndarray, label: str) -> None:
    x = np.sort(data)
    y = np.arange(1, len(x) + 1) / len(x)
    ax.plot(x, y, label=label)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), tight_layout=True)

for s in SEEDS:
    with np.load(f'out/storage_mlp_shuffle/preds_s{s}.npz') as npz:
        plot_ecdf(ax, npz['task_loss'], label=f'seed {s}')

# Post-hoc CRC only

In [ ]:
df_crc = pd.read_csv('out/storage_mlp_shuffle/crc.csv')
df_crc.head()

assert set(df_crc['alpha']) == {0., 1., 2., 5., 10., 15., 20.}
df_crc['alpha'] = df_crc['alpha'].astype(int)

In [ ]:
df_crc_melt = pd.melt(df_crc, id_vars=('seed', 'alpha', 'delta'), value_vars=('task loss', 'cvar', 'lambda'))
df_crc_melt = df_crc_melt.rename({'variable': 'metric'}, axis=1)

fig = plt.figure(figsize=(12, 4), tight_layout=True)

(
    so.Plot(df_crc_melt[df_crc_melt['alpha'] > 0.5], x='alpha', y='value', color='delta')
    .add(so.Dot(), so.Dodge())
    .facet('metric')
    # .scale(x=so.Continuous().tick(at=df_crc_melt['alpha'].unique().tolist()))
    .scale(x=so.Nominal(), color=so.Nominal())
    .share(y=False)
    .on(fig).plot()
)

axes = fig.get_axes()
for ax in axes:
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

In [ ]:
# keys are (alpha, delta, seed)
with open('out/storage_mlp_shuffle/crc_financial_losses.pkl', 'rb') as f:
    crc_financial_losses = pickle.load(f)

rows = []
financial_losses = {}

alpha_plot = 5
delta_plot = 0.9
for s in SEEDS:
    for δ in np.linspace(0, 1, endpoint=True, num=101):
        financial_losses = crc_financial_losses[(alpha_plot, delta_plot, s)]
        row = {
            'seed': s,
            'delta': δ,
            'CVaR': cvar(financial_losses, q=δ.item()),
        }
        rows.append(row)

df_crc_cvar = pd.DataFrame(rows)

# (
#     so.Plot(df_crc_cvar, x='delta', y='CVaR', color='seed')
#     .add(so.Line())
#     .scale(color=so.Nominal())
# )

# avg CVaR over 10 seeds
plt.plot(df_crc_cvar.groupby('delta')['CVaR'].mean())

# E2E CRC

In [ ]:
alphas = [2, 5, 10]
deltas = [0.9, 0.95, 0.99]
seeds = range(10)
lrs = [1e-2, 1e-3, 1e-4, 1e-5]
cols = ('seed', 'alpha', 'delta', 'lr', 'test_mse', 'test_task_loss', 'test_cvar', 'val_task_loss', 'val_lam', 'best_epoch')

rows = []
for a, delta, s, lr in itertools.product(alphas, deltas, seeds, lrs):
    basename = f'a{a:.2f}_delta{delta:.2f}_lr{lr:.2g}_s{s}'
    try:
        with open(f'out/storage_mlp_shuffle/e2ecrc/{basename}.json') as f:
            result = json.load(f)
        assert result['seed'] == s
        assert result['alpha'] == a
        result['alpha'] = int(result['alpha'])
        assert result['lr'] == lr
        rows.append({k: result[k] for k in cols})
    except FileNotFoundError:
        print(f'File not found: {basename}.json')
        continue

df_e2e = pd.DataFrame(rows)
df_e2e.head()

In [ ]:
df_e2e_melt = pd.melt(df_e2e, id_vars=('seed', 'alpha', 'delta', 'lr'), value_vars=('test_task_loss', 'test_cvar', 'val_lam'))
df_e2e_melt = df_e2e_melt.rename({'variable': 'metric'}, axis=1)

fig = plt.figure(figsize=(12, 8), tight_layout=True)

(
    so.Plot(df_e2e_melt, x='alpha', y='value', color='lr')
    .add(so.Dot(), so.Dodge())
    .facet(col='metric', row='delta')
    .scale(x=so.Nominal())
    .scale(color=so.Nominal())
    .share(y=False)
    .on(fig).plot()
)

axes = fig.get_axes()
for ax in axes:
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

In [ ]:
# output columns:
# seed, alpha, delta, lambda, task loss, CVaR
best_hps = df_e2e.groupby(['alpha', 'seed', 'delta'])['val_task_loss'].idxmin().values.tolist()
df_e2e_best = df_e2e.loc[best_hps].reset_index().set_index(['alpha', 'delta', 'seed'])

alpha_plot = 5
delta_plot = 0.9
print(f'best hyperparameters @ alpha={alpha_plot}, delta={delta_plot}:')
display(df_e2e_best.loc[(alpha_plot, delta_plot, slice(None)), :])
for seed in SEEDS:
    lr = df_e2e_best.loc[(alpha_plot, delta_plot, seed), 'lr']
    filename = f'a{alpha_plot:.2f}_delta{delta_plot:.2f}_lr{lr:.2g}_s{seed}.pt'
    print(filename)

df_e2e_best = df_e2e_best.rename(columns={'test_task_loss': 'task loss', 'test_cvar': 'cvar', 'val_lam': 'lambda'})
df_e2e_best = df_e2e_best.reset_index()[['seed', 'alpha', 'delta', 'lambda', 'task loss', 'cvar']]

df_e2e_best.head()

In [ ]:
best_e2e_ckpts = [
    'a5.00_delta0.90_lr0.0001_s0.pt',
    'a5.00_delta0.90_lr0.0001_s1.pt',
    'a5.00_delta0.90_lr1e-05_s2.pt',
    'a5.00_delta0.90_lr0.001_s3.pt',
    'a5.00_delta0.90_lr0.0001_s4.pt',
    'a5.00_delta0.90_lr0.0001_s5.pt',
    'a5.00_delta0.90_lr1e-05_s6.pt',
    'a5.00_delta0.90_lr0.0001_s7.pt',
    'a5.00_delta0.90_lr0.0001_s8.pt',
    'a5.00_delta0.90_lr0.0001_s9.pt',
]
e2e_financial_losses = []
for seed, ckpt_path in enumerate(best_e2e_ckpts):
    rows = run_storage.crc(
        shuffle=True, future_temp=False, label_noise=run_storage.LABEL_NOISE,
        const=run_storage.STORAGE_CONSTS[0], alphas=(alpha_plot,), deltas=(delta_plot,),
        seed=seed, saved_ckpt_fmt=os.path.join('out/storage_mlp_shuffle/e2ecrc', ckpt_path),
        device='cuda')
    e2e_financial_losses.append(rows[0]['financial losses'])

In [ ]:
rows = []
for s in SEEDS:
    financial_losses = e2e_financial_losses[s]
    for δ in np.linspace(0, 1, endpoint=True, num=101):
        rows.append({
            'seed': s,
            'delta': δ,
            'CVaR': cvar(financial_losses, q=δ.item()),
        })

df_e2ecrc_cvar = pd.DataFrame(rows)
# (
#     so.Plot(df_e2ecrc_cvar, x='delta', y='CVaR', color='seed')
#     .add(so.Line())
#     .scale(color=so.Nominal())
# )

# avg CVaR over 10 seeds
plt.plot(df_e2ecrc_cvar.groupby('delta')['CVaR'].mean())

# Fine-tune Task Loss

In [ ]:
alphas = (0, 1, 2, 5, 10, 15, 20)
deltas = (0.8, 0.9, 0.95, 0.99)
seeds = range(10)
lrs = (1e-2, 1e-3, 1e-4, 1e-5, 1e-6)
cols = ('seed', 'alpha', 'lr', 'test_fpr', 'test_fnr', 'val_loss', 'val_lam')

rows = []
for s, lr in itertools.product(seeds, lrs):
    basename = f'lr{lr:.2g}_s{s}'
    try:
        with open(f'out/storage_mlp_shuffle/finetune_taskloss/{basename}.json') as f:
            result = json.load(f)
        assert result['seed'] == s
        assert result['lr'] == lr
        assert set(alphas).issubset(result['alphas'])
        assert set(deltas).issubset(result['deltas'])
        for i, (a, delta) in enumerate(result['alpha_deltas']):
            assert a == int(a)
            a = int(a)
            rows.append({
                'alpha': a,
                'delta': delta,
                'seed': s,
                'lr': lr,
                'val_task_loss': result['val_task_loss'],
                'val_lam': result['val_lams'][i],
                'test_task_loss': result['test_task_losses'][i],
                'test_cvar': result['test_cvars'][i],
            })

    except FileNotFoundError:
        print(f'File not found: {basename}.json')
        continue

df_trainbase = pd.DataFrame(rows)
df_trainbase.head()

In [ ]:
df_trainbase_melt = pd.melt(df_trainbase, id_vars=('seed', 'alpha', 'delta', 'lr'), value_vars=('test_cvar', 'test_task_loss', 'val_lam'))
df_trainbase_melt = df_trainbase_melt.rename({'variable': 'metric'}, axis=1)

# fig = plt.figure(figsize=(12, 10), tight_layout=True)

(
    so.Plot(df_trainbase_melt, x='alpha', y='value', color='lr')
    .add(so.Dot(), so.Dodge(), so.Jitter(.3))
    .facet('metric', row='delta')
    # .scale(x=so.Continuous().tick(at=df_trainbase_melt['alpha'].unique().tolist()))
    .scale(x=so.Nominal())
    .scale(color=so.Nominal())
    .share(y=False)
    .layout(size=(12, 10), engine='tight')
)

In [ ]:
# output columns:
# seed, alpha, lambda, fnr, fpr
best_hps = df_trainbase.groupby(['alpha', 'delta', 'seed'])['val_task_loss'].idxmin().values.tolist()
df_trainbase_best = df_trainbase.loc[best_hps].reset_index().set_index(['alpha', 'delta', 'seed'])

alpha_plot = 5
delta_plot = 0.9
print(f'best hyperparameters @ alpha={alpha_plot}, delta={delta_plot}:')
display(df_trainbase_best.loc[(alpha_plot, delta_plot, slice(None)), :])
for seed in SEEDS:
    lr = df_trainbase_best.loc[(alpha_plot, delta_plot, seed), 'lr']
    filename = f'lr{lr:.2g}_s{seed}.pt'
    print(filename)

df_trainbase_best = df_trainbase_best.rename(columns={'test_task_loss': 'task loss', 'test_cvar': 'cvar', 'val_lam': 'lambda'})
df_trainbase_best = df_trainbase_best.reset_index()[['seed', 'alpha', 'delta', 'lambda', 'cvar', 'task loss']]

df_trainbase_best.head()

In [ ]:
best_ft_ckpts = [
    'lr0.0001_s0.pt',
    'lr1e-06_s1.pt',
    'lr1e-06_s2.pt',
    'lr1e-06_s3.pt',
    'lr0.0001_s4.pt',
    'lr0.0001_s5.pt',
    'lr1e-05_s6.pt',
    'lr1e-06_s7.pt',
    'lr0.0001_s8.pt',
    'lr0.0001_s9.pt',
]
ft_financial_losses = []
for seed, ckpt_path in enumerate(best_ft_ckpts):
    rows = run_storage.crc(
        shuffle=True, future_temp=False, label_noise=run_storage.LABEL_NOISE,
        const=run_storage.STORAGE_CONSTS[0], alphas=(alpha_plot,), deltas=(delta_plot,),
        seed=seed, saved_ckpt_fmt=os.path.join('out/storage_mlp_shuffle/finetune_taskloss', ckpt_path),
        device='cuda')
    ft_financial_losses.append(rows[0]['financial losses'])

In [ ]:
alpha = 5.0
delta = 0.9
rows = []
for s in SEEDS:
    financial_losses = ft_financial_losses[s]
    for δ in np.linspace(0, 1, endpoint=True, num=101):
        rows.append({
            'seed': s,
            'delta': δ,
            'CVaR': cvar(financial_losses, q=δ.item()),
        })

df_ft_cvar = pd.DataFrame(rows)
# (
#     so.Plot(df_ft_cvar, x='delta', y='CVaR', color='seed')
#     .add(so.Line())
#     .scale(color=so.Nominal())
# )

# avg CVaR over 10 seeds
plt.plot(df_ft_cvar.groupby('delta')['CVaR'].mean())

# Combine

In [ ]:
crc_model_name = 'post-hoc conformal\nCVaR control'
trainbase_model_name = 'fine-tune task loss'
e2e_model_name = 'conformal risk training'

df_crc['model'] = crc_model_name
df_trainbase_best['model'] = trainbase_model_name
df_e2e_best['model'] = e2e_model_name
df = pd.concat([df_crc, df_trainbase_best, df_e2e_best], axis=0)
df

In [ ]:
df_melt = pd.melt(df, id_vars=('seed', 'alpha', 'delta', 'model'), value_vars=('task loss', 'cvar', 'lambda'))
df_melt = df_melt.rename({'variable': 'metric'}, axis=1)

In [ ]:
g = sns.catplot(
    data=df_melt[df_melt['alpha'].isin((2,5,10)) & df_melt['delta'].isin((.9, .95, .99))], x='alpha', y='value',
    hue='model', hue_order=(crc_model_name, trainbase_model_name, e2e_model_name),
    col='metric', row='delta', kind='box', sharey=False,
    height=3
)

axes = g.axes.flatten()

# Iterate through the axes and add vertical lines
for ax in axes:
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

In [ ]:
ylabel = r'CVaR${}^\delta[L(\theta, \lambda)]$'
cvar_df = df[df['alpha'].isin((2, 5, 10)) & df['delta'].isin((.9, .95, .99))].rename(columns={
    'cvar': ylabel,
    'alpha': r'$\alpha$',
    'delta': r'$\delta$',
})

g_cvar = sns.catplot(
    data=cvar_df, x=r'$\alpha$', y=ylabel, hue='model',
    hue_order=(crc_model_name, trainbase_model_name, e2e_model_name),
    col=r'$\delta$', sharey=True, kind='box', height=2.8,
)

legend = [c for c in g_cvar.figure.get_children() if isinstance(c, matplotlib.legend.Legend)][0]
assert isinstance(legend, matplotlib.legend.Legend)
legend.set_loc('outside right center')
for text in legend.get_texts():
    if text.get_text() == e2e_model_name:
        text.set_fontweight('bold')

# Iterate through the axes and add vertical lines
for ax in g_cvar.axes.flatten():
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

g_cvar.figure.set_layout_engine('constrained')
# g_cvar.tight_layout(pad=0, w_pad=1.08)

g_cvar.figure.savefig(os.path.join(savedir, 'storage_cvar.pdf'), pad_inches=0)
g_cvar.figure.savefig(os.path.join(savedir, 'storage_cvar.png'), pad_inches=0, dpi=300)

In [ ]:
# lambdas plot
ylabel = r'$\lambda$'
lambda_df = df[df['alpha'].isin((2, 5, 10)) & df['delta'].isin((.9, .95, .99))].rename(columns={
    'lambda': ylabel,
    'alpha': r'$\alpha$',
    'delta': r'$\delta$',
})

g_lambda = sns.catplot(
    data=lambda_df, x=r'$\alpha$', y=ylabel, hue='model',
    hue_order=(crc_model_name, trainbase_model_name, e2e_model_name),
    col=r'$\delta$', sharey=True, kind='box', height=2.8,
)

legend = [c for c in g_lambda.figure.get_children() if isinstance(c, matplotlib.legend.Legend)][0]
assert isinstance(legend, matplotlib.legend.Legend)
legend.set_loc('outside right center')
for text in legend.get_texts():
    if text.get_text() == e2e_model_name:
        text.set_fontweight('bold')

# Iterate through the axes and add vertical lines
for ax in g_lambda.axes.flatten():
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

g_lambda.figure.set_layout_engine('constrained')
# g_lambda.tight_layout(pad=0, w_pad=1.08)

g_lambda.figure.savefig(os.path.join(savedir, 'storage_lambda.pdf'), pad_inches=0)
g_lambda.figure.savefig(os.path.join(savedir, 'storage_lambda.png'), pad_inches=0, dpi=300)

In [ ]:
comp_df = (
    df[df['alpha'].isin((2,5,10)) & df['delta'].isin((.9, .95, .99))]
    .set_index(['seed', 'alpha', 'delta', 'model'])
    .unstack('model')
)

conf_rel_improvement = (
    (comp_df['task loss'][e2e_model_name] - comp_df['task loss'][crc_model_name])
    / comp_df['task loss'][crc_model_name] * 100
)
fine_rel_improvement = (
    (comp_df['task loss'][trainbase_model_name] - comp_df['task loss'][crc_model_name])
    / comp_df['task loss'][crc_model_name] * 100
)
# crc_rel_improvement = comp_df['task loss'][crc_model_name] * 0.
rel_df = pd.concat([
    conf_rel_improvement.rename(e2e_model_name),
    fine_rel_improvement.rename(trainbase_model_name),
    # crc_rel_improvement.rename(crc_model_name),
], axis=1)

display(rel_df.head())

display(rel_df[e2e_model_name].groupby(['alpha', 'delta']).mean())

In [ ]:
ylabel = '% relative increase\nin mean profit'
rel_df_melt = pd.melt(rel_df.reset_index(), id_vars=('seed', 'alpha', 'delta'), value_vars=(e2e_model_name, trainbase_model_name))
rel_df_melt = rel_df_melt.rename({
    'variable': 'model',
    'value': ylabel,
    'alpha': r'$\alpha$',
    'delta': r'$\delta$',
}, axis=1)

In [ ]:
g_rel = sns.catplot(
    data=rel_df_melt, x=r'$\alpha$', y=ylabel,
    hue='model', hue_order=(trainbase_model_name, e2e_model_name),
    palette=['tab:orange', 'tab:green'],
    col=r'$\delta$', kind='box', sharey=True,
    height=2.8
)

legend = [c for c in g_rel.figure.get_children() if isinstance(c, matplotlib.legend.Legend)][0]
assert isinstance(legend, matplotlib.legend.Legend)
legend.set_loc('outside right center')
for text in legend.get_texts():
    if text.get_text() == e2e_model_name:
        text.set_fontweight('bold')

# Iterate through the axes and add vertical lines
for ax in g_rel.axes.flatten():
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

g_rel.figure.set_layout_engine('constrained')
# g_rel.tight_layout(pad=0, w_pad=1.08)

g_rel.figure.savefig(os.path.join(savedir, 'storage_rel.pdf'), pad_inches=0)
g_rel.figure.savefig(os.path.join(savedir, 'storage_rel.png'), pad_inches=0, dpi=300)

In [ ]:
df_pretrained['model'] = 'pretrained'
df_e2ecrc_cvar['model'] = e2e_model_name
df_crc_cvar['model'] = crc_model_name
df_ft_cvar['model'] = trainbase_model_name
df_cvar = pd.concat([df_pretrained, df_e2ecrc_cvar, df_crc_cvar, df_ft_cvar], axis=0)

In [ ]:
df_avgcvar = df_cvar.groupby(['model', 'delta'])['CVaR'].mean().to_frame()
(
    so.Plot(df_avgcvar, x='delta', y='CVaR', color='model')
    .add(so.Line())
    .scale(color=so.Nominal())
)